# Fine-tuning with only df4(all_agree) using Financial phrasebank dataset

In [1]:
import numpy as np
import pandas as pd
import transformers
import torch
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import load_dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset, load_metric

In [2]:
torch.__version__, transformers.__version__

('1.13.1+cu117', '4.27.2')

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060


In [4]:
#load train data

df11 = load_dataset('financial_phrasebank', 'sentences_allagree')
df1 = pd.DataFrame(df11)

Found cached dataset financial_phrasebank (C:/Users/gmleh/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# 보기 편하게 만들기

df1['sentence'] = df1['train'].apply(lambda x: x.get('sentence', None) if isinstance(x, dict) else None)
df1['label'] = df1['train'].apply(lambda x: x.get('label', None) if isinstance(x, dict) else None)
df1 = df1[['sentence', 'label']]

df1.head()

,sentence,label
0,"According to Gran , the company has no plans t...",1
1,"For the last quarter of 2010 , Componenta 's n...",2
2,"In the third quarter of 2010 , net sales incre...",2
3,Operating profit rose to EUR 13.1 mn from EUR ...,2
4,"Operating profit totalled EUR 21.1 mn , up fro...",2


In [6]:
def custom_metrics(eval_pred):
    f1_metric     = load_metric("f1")
    acc_metric    = load_metric("accuracy")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    f1 = f1_metric.compute(predictions = predictions, references = labels, average = "weighted")["f1"]
    accuracy = acc_metric.compute(predictions = predictions, references = labels)["accuracy"]

    return {"f1": f1, "accuracy": accuracy}

In [7]:
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=128)

# Fine-tuning based on the number of fine-tuning training datasets

In [ ]:
# train 90 / test 10

models = ['lukecarlate/SECBERT_Num', 'lukecarlate/BERT_Num', 'lukecarlate/Araci_Num', 'lukecarlate/Yang_Num',
         'ProsusAI/finbert', 'bert-base-uncased', 'yiyanghkust/finbert-pretrain', 'nlpaueb/sec-bert-base']

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# for df_name, (df_train, df_test) in data_frames.items():
for model_name in models:
    fold = 0
    for train_index, test_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('train90test10_result.csv')

In [ ]:
# train 80 / test 20

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for model_name in models:
    fold = 0
    for train_index, test_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('train80test20_result.csv')

In [ ]:
# train 50 / test 50

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for model_name in models:
    fold = 0
    for train_index, test_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('train50test50_result.csv')

In [ ]:
# train 20 / test 80

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for model_name in models:
    fold = 0
    for test_index, train_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('train20test80_result.csv')

In [ ]:
# train 10 / test 90

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for model_name in models:
    fold = 0
    for test_index, train_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('train10test90_result.csv')

# Increase the number of post-training training datasets

In [ ]:
models = ['lukecarlate/Araci_Num_2', 'lukecarlate/Araci_Num_3', 'lukecarlate/Araci_Num_4', 'lukecarlate/Araci_Num_5']

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 

for model_name in models:
    fold = 0
    for train_index, test_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        # Use the tokenize_function
        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        # Set the format for pytorch
        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('variation_of_training_set_result8020_335588.csv')

In [ ]:
# train : test = 5 : 5

models = ['lukecarlate/Araci_Num_2', 'lukecarlate/Araci_Num_3', 'lukecarlate/Araci_Num_4', 'lukecarlate/Araci_Num_5']

args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42) 

for model_name in models:
    fold = 0
    for train_index, test_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        # Use the tokenize_function
        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        # Set the format for pytorch
        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('variation_of_training_set_result5050_335588.csv')

In [9]:
# train 20 / test 80

models = ['lukecarlate/Araci_Num_2', 'lukecarlate/Araci_Num_3', 'lukecarlate/Araci_Num_4', 'lukecarlate/Araci_Num_5']


args = TrainingArguments(
    output_dir="/textmining/result",          
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,                  
    per_device_train_batch_size=64,        
    per_device_eval_batch_size=64,         
    num_train_epochs=6,                     
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_steps=10,
    logging_dir='/textmining/log',
    save_steps=10,
    warmup_ratio=0.2,
)

results = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for model_name in models:
    fold = 0
    for test_index, train_index in kf.split(df1['sentence'], df1['label']):
        fold += 1
        print(f"Training {model_name} - Fold {fold}")

        df_train_fold, df_test_fold = df1.iloc[train_index], df1.iloc[test_index]

        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        df_train_fold = Dataset.from_pandas(df_train_fold)
        df_test_fold = Dataset.from_pandas(df_test_fold)

        df_train_fold = df_train_fold.map(tokenize_function, batched=True)
        df_test_fold = df_test_fold.map(tokenize_function, batched=True)

        df_train_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
        df_test_fold.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=df_train_fold,
            eval_dataset=df_test_fold,
            compute_metrics=custom_metrics
        )

        trainer.train()
        metrics = trainer.predict(df_test_fold).metrics

        results.append({
            'model': model_name,
            'fold': fold,
            'f1': metrics['test_f1'],
            'accuracy': metrics['test_accuracy']
        })
    
df_results = pd.DataFrame(results)

means = df_results.groupby('model')[['f1', 'accuracy']].mean().reset_index()
means.to_csv('variation_of_training_set_result2080_335588.csv')

Training lukecarlate/Araci_Num_11 - Fold 1


Some weights of the model checkpoint at lukecarlate/Araci_Num_11 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lukecarlate/Araci_Nu

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

C:\Users\gmleh\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Some weights of the model checkpoint at lukecarlate/Araci_Num_11 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lukecarlate/Araci_Nu

Map:   0%|          | 0/452 [00:00<?, ? examples/s]

Map:   0%|          | 0/1812 [00:00<?, ? examples/s]

C:\Users\gmleh\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.880983,0.602625,0.685430
2,1.034300,0.472902,0.881648,0.887417
3,0.556300,0.284930,0.932682,0.933775
4,0.274500,0.214899,0.934541,0.935430
5,0.148100,0.188894,0.934646,0.935430
6,0.148100,0.180769,0.939911,0.940397
